Les LSTM sont un type de réseau de neurones récurrents (RNN) qui sont particulièrement efficaces pour traiter les séquences de données, comme les séquences de texte ou de temps.

La cellule LSTM a trois portes principales qui contrôlent le flux d'informations : la porte d'oubli (forget gate), la porte d'entrée (input gate) et la porte de sortie (output gate).

1. **Porte d'oubli (Forget Gate)** :
   $$
   f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)
   $$
   Cette porte détermine quelles informations de l'état de la cellule précédente doivent être oubliées.

2. **Porte d'entrée (Input Gate)** :
   $$
   i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)
   $$
   $$
   \tilde{C}_t = \tanh(W_C \cdot [h_{t-1}, x_t] + b_C)
   $$
   Cette porte détermine quelles nouvelles informations doivent être stockées dans l'état de la cellule.

3. **Mise à jour de l'état de la cellule** :
   $$
   C_t = f_t \cdot C_{t-1} + i_t \cdot \tilde{C}_t
   $$
   Cela met à jour l'état de la cellule en combinant les anciennes informations avec les nouvelles.

4. **Porte de sortie (Output Gate)** :
   $$
   o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o)
   $$
   $$
   h_t = o_t \cdot \tanh(C_t)
   $$
   Cette porte détermine quelle partie de l'état de la cellule doit être exposée en tant que sortie.

Dans ces équations, $x_t$ représente l'entrée à l'instant $t$, $h_t$ est la sortie à l'instant $t$, $C_t$ est l'état de la cellule à l'instant $t$, et $\sigma$ représente la fonction sigmoïde. $W$ et $b$ sont les poids et les biais qui sont appris pendant l'entraînement du réseau.

Les LSTM sont puissants car ils permettent de conserver et d'utiliser des informations sur de longues séquences, ce qui les rend particulièrement utiles dans des tâches comme la traduction automatique, la génération de texte et d'autres domaines liés au traitement de la langue naturelle.

### CustomLSTMCell

Cette classe représente une seule cellule LSTM. Chaque cellule LSTM a plusieurs portes (porte d'oubli, porte d'entrée, cellule candidate et porte de sortie) qui régulent le flux d'informations à travers la cellule.

1. **`__init__`** : Dans le constructeur, nous définissons les composants de la cellule LSTM. Chaque porte a deux couches linéaires : `W_*` qui traite les entrées et `U_*` qui traite l'état de la cellule précédent.

2. **`forward`** : Cette méthode définit comment les opérations sont effectuées lors du passage d'une entrée à travers la cellule LSTM.

    - On initialise l'état caché `h_t` et l'état de la cellule `c_t` s'ils ne sont pas fournis.
    - On calcule les portes d'oubli `f_t`, d'entrée `i_t`, de cellule candidate `g_t` et de sortie `o_t` en utilisant les couches linéaires et des fonctions d'activation appropriées.
    - On met à jour l'état de la cellule `c_t`.
    - On met à jour l'état caché `h_t`.
    - Enfin, on retourne `h_t` et `c_t`.

In [ ]:
import torch
import torch.nn as nn
import numpy as np

In [ ]:

class CustomLSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(CustomLSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        # Portes d'oubli
        self.W_f = nn.Linear(input_size + hidden_size, hidden_size)
        self.U_f = nn.Linear(hidden_size, hidden_size)

        # Portes d'entrée
        self.W_i = nn.Linear(input_size + hidden_size, hidden_size)
        self.U_i = nn.Linear(hidden_size, hidden_size)

        # Cellule candidate
        self.W_c = nn.Linear(input_size + hidden_size, hidden_size)
        self.U_c = nn.Linear(hidden_size, hidden_size)

        # Porte de sortie
        self.W_o = nn.Linear(input_size + hidden_size, hidden_size)
        self.U_o = nn.Linear(hidden_size, hidden_size)

    def forward(self, x, init_states=None):
        bs, _ = x.size()

        h_t, c_t = (torch.zeros(self.hidden_size).to(x.device),
                    torch.zeros(self.hidden_size).to(x.device)) if init_states is None else init_states

        h_t, c_t = h_t.view(bs, -1), c_t.view(bs, -1)

        # Calcul des portes d'oubli
        f_t = torch.sigmoid(self.W_f(torch.cat([x, h_t], dim=1)) + self.U_f(c_t))

        # Calcul des portes d'entrée
        i_t = torch.sigmoid(self.W_i(torch.cat([x, h_t], dim=1)) + self.U_i(c_t))

        # Calcul de la cellule candidate
        g_t = torch.tanh(self.W_c(torch.cat([x, h_t], dim=1)) + self.U_c(c_t))

        # Mise à jour de l'état de la cellule
        c_t = f_t * c_t + i_t * g_t

        # Calcul de la sortie
        o_t = torch.sigmoid(self.W_o(torch.cat([x, h_t], dim=1)) + self.U_o(c_t))

        # Mise à jour de l'état caché
        h_t = o_t * torch.tanh(c_t)

        return h_t, c_t



### CustomLSTM

Cette classe encapsule le fonctionnement d'une séquence de cellules LSTM. Elle itère sur les éléments de la séquence en utilisant la cellule LSTM que nous avons définie précédemment.

1. **`__init__`** : Le constructeur de la classe initialise les dimensions de l'entrée et de l'état caché, puis crée une instance de `CustomLSTMCell`.

2. **`forward`** : Cette méthode définit comment les opérations sont effectuées lors du passage de l'ensemble d'une séquence à travers le LSTM.

    - On initialise l'état caché `h_t` et l'état de la cellule `c_t` à zéro.
    - On itère sur les éléments de la séquence (`seq_len`) en appliquant la cellule LSTM sur chaque élément. On met à jour `h_t` et `c_t` à chaque itération.
    - On retourne la sortie `h_t`.

In [ ]:


class CustomLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(CustomLSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.lstm_cell = CustomLSTMCell(input_size, hidden_size)

    def forward(self, x):
        bs, seq_len, _ = x.size()

        h_t, c_t = (torch.zeros(self.hidden_size).to(x.device),
                    torch.zeros(self.hidden_size).to(x.device))

        h_t, c_t = h_t.view(1,-1).repeat(bs,1), c_t.view(1,-1).repeat(bs,1)

        h_t_list = []
        for t in range(seq_len):
            h_t, c_t = self.lstm_cell(x[:, t, :], (h_t, c_t))
            h_t_list.append(h_t.unsqueeze(1))

        h_t = torch.cat(h_t_list, dim=1)

        return h_t



## Utilisation

Pour utiliser cette implémentation personnalisée de LSTM, vous pouvez créer une instance de `CustomLSTM` et l'utiliser dans le processus d'entraînement de votre modèle. Assurez-vous de fournir les dimensions d'entrée et de l'état caché appropriées lors de la création de l'instance.


In [ ]:
# Paramètres
input_size = 5
hidden_size = 8
num_layers = 2
output_size = 1
num_samples = 100
sequence_length = 10

In [ ]:
# Générer des séquences aléatoires
X = np.random.rand(num_samples, sequence_length, input_size)
y = np.sum(X, axis=2)  # La sortie est la somme des entrées

# Convertir les données en tenseurs PyTorch
X = torch.Tensor(X)
y = torch.Tensor(y).unsqueeze(-1)

In [ ]:


# Initialiser le modèle et définir la fonction de coût et l'optimiseur
model = CustomLSTM(input_size, hidden_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Entraînement
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(X)

    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Époque [{epoch+1}/100], Perte : {loss.item():.4f}')




/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([100, 10, 1])) that is different to the input size (torch.Size([100, 10, 8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Époque [10/100], Perte : 6.4849
Époque [20/100], Perte : 6.3146
Époque [30/100], Perte : 6.1468
Époque [40/100], Perte : 5.9610
Époque [50/100], Perte : 5.7067
Époque [60/100], Perte : 5.3495
Époque [70/100], Perte : 4.9441
Époque [80/100], Perte : 4.5781
Époque [90/100], Perte : 4.2935
Époque [100/100], Perte : 4.0701


In [ ]:


# Définir une classe pour le LSTM sous pytorch
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()

        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        out = self.fc(out[:, -1, :])
        return out

model=LSTM()